In [166]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from PIL import *
import os
import warnings
import tensorflow as tf
warnings.filterwarnings("ignore")
from sklearn.metrics import *

In [167]:
tumor_dataset_path = os.getcwd() + "\data"

image_files = []
labels = []
ctr = 0

lab = []
for i in os.listdir(tumor_dataset_path):
    lab.append(i)
lab = sorted(lab)
    
for i in lab:
    k = tumor_dataset_path + "\\" + i + "\\"
    for j in os.listdir(k):
        image_files.append(k + j)
        labels.append(ctr)
    ctr += 1
image_files = np.array(image_files)
labels = np.array(labels)
labels_1 = labels.copy()

In [168]:
images = []
images_1 = []

for i in image_files:
    image_original = Image.open(i)
    image_grayscale = image_original.convert("L")
    image_resized = image_grayscale.resize((64,64))
    image_array = np.array(image_resized) / 255.0
    image_array = image_array.reshape((64,64,1))
    images.append(image_array)

    image_resized_1 = image_grayscale.resize((256,256))
    image_array_1 = np.array(image_resized_1) / 255.0
    image_array_1 = image_array_1.reshape((256,256,1))
    images_1.append(image_array_1)

    
images = np.array(images)
images_1 = np.array(images_1)

In [169]:
images[0].shape

(64, 64, 1)

In [170]:
images_1[0].shape

(256, 256, 1)

In [171]:
image_positions = list(range(len(images)))
np.random.shuffle(image_positions)
image_labels_shuffled = labels[image_positions]
image_inputs_shuffled = images[image_positions]

image_positions_1 = list(range(len(images_1)))
np.random.shuffle(image_positions_1)
image_labels_shuffled_1 = labels_1[image_positions_1]
image_inputs_shuffled_1 = images_1[image_positions_1]

In [172]:
number_of_images = len(image_inputs_shuffled)

div_ratio = 0.25

test_size = round(number_of_images * div_ratio)
train_val_size = number_of_images - test_size
val_test_threshold = train_val_size

val_size = round(train_val_size * div_ratio)
train_size = train_val_size - val_size
train_val_threshold = train_size


number_of_images_1 = len(image_inputs_shuffled_1)

div_ratio_1 = 0.2

test_size_1 = round(number_of_images_1 * div_ratio_1)
train_val_size_1 = number_of_images_1 - test_size_1
val_test_threshold_1 = train_val_size_1

val_size_1 = round(train_val_size_1 * div_ratio_1)
train_size_1 = train_val_size_1 - val_size_1
train_val_threshold_1 = train_size_1

In [173]:
print(val_test_threshold)
print(train_val_threshold)

194
146


In [174]:
print(val_test_threshold_1)
print(train_val_threshold_1)

207
166


In [175]:
X_train = image_inputs_shuffled[:train_val_threshold]
y_train = image_labels_shuffled[:train_val_threshold]

X_val = image_inputs_shuffled[train_val_threshold:val_test_threshold]
y_val = image_labels_shuffled[train_val_threshold:val_test_threshold]

X_test = image_inputs_shuffled[val_test_threshold:]
y_test = image_labels_shuffled[val_test_threshold:]


X_train_1 = image_inputs_shuffled_1[:train_val_threshold_1]
y_train_1 = image_labels_shuffled_1[:train_val_threshold_1]

X_val_1 = image_inputs_shuffled_1[train_val_threshold_1:val_test_threshold_1]
y_val_1 = image_labels_shuffled_1[train_val_threshold_1:val_test_threshold_1]

X_test_1 = image_inputs_shuffled_1[val_test_threshold_1:]
y_test_1 = image_labels_shuffled_1[val_test_threshold_1:]

In [176]:
print(X_train.shape[0],X_val.shape[0],X_test.shape[0])
print(X_train_1.shape[0],X_val_1.shape[0],X_test_1.shape[0])

146 48 65
166 41 52


In [177]:
model = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation="relu", input_shape=(64,64,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(units=64, activation="relu"),
    tf.keras.layers.Dense(units=1, activation="sigmoid")  

])

In [178]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.binary_crossentropy, metrics=tf.keras.metrics.binary_accuracy)
model.fit(X_train,y_train,epochs=10,shuffle=True,validation_data=(X_val,y_val),verbose=1,callbacks=tf.keras.callbacks.EarlyStopping(patience=2))

Epoch 1/10
5/5 [==============================] - 2s 80ms/step - loss: 0.6908 - binary_accuracy: 0.5685 - val_loss: 0.6713 - val_binary_accuracy: 0.6042
Epoch 2/10
5/5 [==============================] - 0s 30ms/step - loss: 0.6905 - binary_accuracy: 0.5959 - val_loss: 0.6773 - val_binary_accuracy: 0.6042
Epoch 3/10
5/5 [==============================] - 0s 24ms/step - loss: 0.6790 - binary_accuracy: 0.5685 - val_loss: 0.6626 - val_binary_accuracy: 0.6042
Epoch 4/10
5/5 [==============================] - 0s 24ms/step - loss: 0.6697 - binary_accuracy: 0.5685 - val_loss: 0.6550 - val_binary_accuracy: 0.6458
Epoch 5/10
5/5 [==============================] - 0s 24ms/step - loss: 0.6560 - binary_accuracy: 0.5822 - val_loss: 0.6331 - val_binary_accuracy: 0.6667
Epoch 6/10
5/5 [==============================] - 0s 25ms/step - loss: 0.6257 - binary_accuracy: 0.8082 - val_loss: 0.6045 - val_binary_accuracy: 0.7292
Epoch 7/10
5/5 [==============================] - 0s 24ms/step - loss: 0.5763 - bi

In [179]:
y_pred.round()

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [180]:
y_pred = model.predict(X_test,verbose=0).round()
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", round(accuracy,2))

Accuracy: 0.71


In [181]:
test_path = os.getcwd() + "\\test\\"
test_images = []
test_images_1 = []
test_labels = []
test_labels_1 = []
gg = 00
oo = []
for i in os.listdir(test_path):
    image_original = Image.open(test_path + i)
    image_grayscale = image_original.convert("L")
    
    image_resized = image_grayscale.resize((64,64))
    image_array = np.array(image_resized) / 255.0
    image_array = image_array.reshape((64,64,1))
    test_images.append(image_array)


    image_resized_1 = image_grayscale.resize((256,256))
    image_array_1 = np.array(image_resized_1) / 255.0
    image_array_1 = image_array_1.reshape((256,256,1))
    test_images_1.append(image_array_1)

test_images = np.array(test_images)
test_images_1 = np.array(test_images_1)

In [182]:
test_images[0].shape

(64, 64, 1)

In [183]:
test_images_1[0].shape

(256, 256, 1)

In [184]:
test_labels = model.predict(test_images,verbose=0).round()

In [185]:
test_labels

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [186]:
model_1 = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation="relu", input_shape=(256,256,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),


    tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),


    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(units=128, activation="relu"),
    tf.keras.layers.Dense(units=1, activation="sigmoid")

])

In [187]:
model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.binary_crossentropy, metrics=tf.keras.metrics.binary_accuracy)
model_1.fit(X_train_1,y_train_1,epochs=75,shuffle=True,validation_data=(X_val_1,y_val_1),verbose=1,callbacks=tf.keras.callbacks.EarlyStopping(patience=4))

Epoch 1/75
6/6 [==============================] - 3s 216ms/step - loss: 0.6771 - binary_accuracy: 0.5964 - val_loss: 0.6534 - val_binary_accuracy: 0.5610
Epoch 2/75
6/6 [==============================] - 1s 150ms/step - loss: 0.6195 - binary_accuracy: 0.6928 - val_loss: 0.5778 - val_binary_accuracy: 0.7561
Epoch 3/75
6/6 [==============================] - 1s 139ms/step - loss: 0.6080 - binary_accuracy: 0.6747 - val_loss: 0.5273 - val_binary_accuracy: 0.7805
Epoch 4/75
6/6 [==============================] - 1s 138ms/step - loss: 0.5616 - binary_accuracy: 0.7289 - val_loss: 0.5217 - val_binary_accuracy: 0.7561
Epoch 5/75
6/6 [==============================] - 1s 138ms/step - loss: 0.5684 - binary_accuracy: 0.7229 - val_loss: 0.4795 - val_binary_accuracy: 0.7561
Epoch 6/75
6/6 [==============================] - 1s 136ms/step - loss: 0.5299 - binary_accuracy: 0.7349 - val_loss: 0.4568 - val_binary_accuracy: 0.8049
Epoch 7/75
6/6 [==============================] - 1s 137ms/step - loss: 0.51

In [188]:
y_pred_1 = model_1.predict(X_test_1,verbose=1).round()
accuracy_1 = accuracy_score(y_test_1, y_pred_1)
print("Accuracy:", round(accuracy_1,2))

2/2 [==============================] - 0s 44ms/step
Accuracy: 0.83


In [189]:
test_labels_1 = model_1.predict(test_images_1,verbose=0).round()
test_labels_1

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.]], dtype=float32)